# Chi-Squared Calculation Comparison

This notebook compares the published Chi-Squared values from DES against the Chi-Squared value calculated by SNCosmo using the published fit parameters.

In [ ]:
import sys
from warnings import warn

import sncosmo
from astropy.table import Table
from matplotlib import pyplot as plt

sys.path.insert(0, '../')
from analysis_pipeline.data_access import des

param_names = sncosmo.Model(source='salt2').param_names

# Calculating Chi-Squared

In [ ]:
def calc_chi_sq(modelcov=True):
    """Calculate the Chi-Squared value for each target
    
    Args:
        modelcov (bool): Whether to use the model covariance 
          when calculating chi-squared (Default: True)
          
    Returns:
        An astropy table with chi-squared values and fit parameters
    """
    
    # Create duplicate columns with new names for convenience
    des.master_table['z'] = des.master_table['zCMB']
    des.master_table['zERR'] = des.master_table['zCMBERR']
    des.master_table['t0'] = des.master_table['PKMJD']
    des.master_table['t0ERR'] = des.master_table['PKMJDERR']

    # Create output Table
    col_names = ['cid', 'NDOF', 'FITCHI2', 'FITPROB', 'SNCOSMOCHI2']
    col_names.extend(param_names)
    col_names.extend((p + 'ERR' for p in param_names))
    out_table = Table(names=col_names)
    
    # Poplulat table
    for target in des.master_table:
        try:
            data = des.get_input_for_id(target['CIDint'])

        except:
            continue

        salt2 = sncosmo.Model(source='salt2')
        fit_params = {param: target[param] for param in salt2.param_names}
        salt2.set(**fit_params)

        try:
            chi_sq = sncosmo.chisq(data, salt2, modelcov=modelcov)

        except ValueError as e:
            warn(f'Failed calculation for {target["CID"]}: {e}')
                 
        new_row = list(target['CIDint', 'NDOF', 'FITCHI2', 'FITPROB'])
        new_row.append(chi_sq)
        new_row.extend(fit_params.values())
        for p in param_names:
                 new_row.append(target[p + 'ERR'])
        out_table.add_row(new_row)

    return out_table
                 


In [ ]:
chi_sq_cov = calc_chi_sq(True)
chi_sq_nocov = calc_chi_sq(False)

chi_sq_cov.show_in_notebook(display_length=5)

# Comparing Chi-Squared Values

We compare Chi-Squared values calculated with and without covariance

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 5))

for axis, data in zip(axes, [chi_sq_cov, chi_sq_nocov]):
    x, y = data['FITCHI2'], data['SNCOSMOCHI2']
    axis.scatter(x, y)
    axis.plot((0, 20000), (0, 20000), linestyle='--', alpha=.5, color='grey')
    
    axis.set_xlabel('Published Chi2')
    axis.set_xlim(0, max(x))
    axis.set_ylim(0, max(y))

axes[0].set_ylabel('SNCosmo Chi2')
axes[0].set_title('Covariance')
axes[1].set_title('No Covariance')

plt.show()

# Correlations with Parameters

We expect there to be no correlation between the chi-squared residuals, but we check anyways.

In [ ]:
residual = chi_sq_cov['FITCHI2'] - chi_sq_cov['SNCOSMOCHI2']
fig, axes = plt.subplots(2, 3, figsize=(16, 8))

for axis, param in zip(axes.flatten(), param_names):
    axis.scatter(chi_sq_cov[param], residual)
    axis.set_xlim(min(chi_sq_cov[param]), max(chi_sq_cov[param]))
    axis.set_title(param)
    
axes[1, 2].scatter(chi_sq_cov['FITPROB'], residual)
axes[1, 2].set_xlim(min(chi_sq_cov['FITPROB']), max(chi_sq_cov['FITPROB']))
axes[1, 2].set_title('FITPROB')

axes[0, 0].set_ylabel('Residual Chi-Squared')
axes[1, 1].set_xlabel('Parameter Value')
plt.show()


In [ ]:
residual = chi_sq_cov['FITCHI2'] - chi_sq_cov['SNCOSMOCHI2']
fig, axes = plt.subplots(2, 3, figsize=(16, 8))

for axis, param in zip(axes.flatten(), param_names):
    axis.scatter(chi_sq_cov[param + 'ERR'], residual)
    axis.set_xlim(min(chi_sq_cov[param + 'ERR']), max(chi_sq_cov[param + 'ERR']))
    axis.set_title(param + 'ERR')

axes[0, 0].set_ylabel('Residual Chi-Squared')
axes[1, 1].set_xlabel('Parameter Value')
plt.show()

